# Module Import

In [ ]:
# Crawling
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta

# Basic
import pandas as pd
import numpy as np
import openpyxl

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Language Compatibility
from matplotlib import font_manager, rc

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

# Define Crawling Function

In [ ]:
def yanolja_crawl(url):
    global df # 전역 변수를 지역 범위에서 적용
    driver.get(url)

    hotel = driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf h1').text
    hotel = hotel.replace("[★숙박대전] ", "")

    # 리뷰 페이지 클릭 (Click review page)
    driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf > a').click()

    # 무한 스크롤
    import time
    time.sleep(1)

    SCROLL_PAUSE_TIME = 1.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break

        last_height = new_height

    # 스크롤 끝나고 페이지 elements 가져오기
    raw = driver.page_source # 페이지의 elements 모두 가져오기
    html = BeautifulSoup(raw, "html.parser")

    # 리뷰 텍스트 선택자: div.item-wrap
    reviews = html.select("div.item-wrap")

    columns = ['hotel', 'star', 'review', 'date']

    for r in reviews:
        review = r.select_one("p")
        date = r.select_one('time')

        stars = r.select('.container.score')
        for s in stars:
            star_count = 5 - len(s.select(".ico.gray"))

        df = df.append(pd.DataFrame([[hotel, star_count, review.text, date.text]], columns = columns),
                       ignore_index=True)
    
def change_date(x):
    now = datetime.datetime.today()
    today = now.strftime("%Y. %m. %d")
    yesterday = (now - timedelta(days=1)).strftime("%Y. %m. %d")
    _2days_ago = (now - timedelta(days = 2)).strftime("%Y. %m. %d")
    _3days_ago = (now - timedelta(days = 3)).strftime("%Y. %m. %d")
    hour = int(datetime.datetime.today().strftime("%H"))
    if "시간 전" in x:
        x = int(x.replace("시간 전", ""))
        if hour - x > 0: return today
        else: return yesterday
    elif "일 전" in x:
        if "1" in x: return yesterday
        elif "2" in x: return _2days_ago
        else: return _3days_ago
    else: return x

# Crawling

In [ ]:
url_list = [ "https://www.yanolja.com/hotel/3001317", # 라까사 호텔 서울
             "https://www.yanolja.com/guest-house/10040389", # 라까사 호텔 서울_언바운디 -> 390~393
             "https://www.yanolja.com/hotel/3016071", # 라까사 호텔 광명
             "https://www.yanolja.com/guest-house/10041118", # 라까사 호텔 광명_언바운디 -> 1386~1390
             "https://www.yanolja.com/hotel/3000613", # 코리아나 호텔
             "https://www.yanolja.com/guest-house/10040099", # 호텔 코리아나_언바운디 -> 1877~1878 / 호텔 코리아나
             "https://www.yanolja.com/hotel/3009495", # 홍대 호텔 나인브릭
             "https://www.yanolja.com/guest-house/10040690", # 호텔 나인브릭_언바운디 -> 3784 / 호텔 나인브릭
             "https://www.yanolja.com/hotel/3000720", # 더 리버사이드 호텔
             "https://www.yanolja.com/guest-house/10040173", # 호텔 더 리버사이드_언바운디 -> 4525~4527 / 호텔 더 리버사이드
             "https://www.yanolja.com/guest-house/1000108371", # 평창 고려궁 한옥호텔_언바운디 -> 4528~4537
             "https://www.yanolja.com/hotel/10040002", # 부산 스튜디오202
             "https://www.yanolja.com/guest-house/1000113591", # 부산 스튜디오202_언바운디 -> 4545~4586 / 광안리 스튜디오 202
             "https://www.yanolja.com/guest-house/1000111730" # 서귀포 일레인 호텔 _언바운디 -> 4587~4628
           ]

columns = ['hotel', 'star', 'review', 'date']
df = pd.DataFrame(columns = columns) # 데이터프레임 생성

In [ ]:
path = 'C:/Users/zyunhyeok/Desktop/자료/chromedriver/chromedriver.exe'
driver = webdriver.Chrome(path)

for url in url_list:
    yanolja_crawl(url)

# driver.close()

C:\Users\zyunhyeok\AppData\Local\Temp\ipykernel_23856\64086834.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
C:\Users\zyunhyeok\AppData\Local\Temp\ipykernel_23856\774128163.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  hotel = driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf h1').text
C:\Users\zyunhyeok\AppData\Local\Temp\ipykernel_23856\774128163.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf > a').click()


In [ ]:
df['real_date'] = df['date'].apply(change_date)
df = df.rename(columns={'real_date': 'date'})

In [ ]:
df.to_csv("../data/crawling/crawling.csv")
df.to_excel("../data/crawling/crawling.xlsx")

# Define Function

In [2]:
import stanza

nlp = stanza.Pipeline('ko')

def extract_nouns(text):
    doc = nlp(text)
    for sentence in doc.sentences:
        for word in sentence.words:
            lemma = word.lemma.split('+')
            xpos = word.xpos.split('+')
            for lem, pos in zip(lemma, xpos):
                if pos.startswith('n'):
                    yield lem

2022-02-14 14:20:23 INFO: Loading these models for language: ko (Korean):
| Processor | Package |
-----------------------
| tokenize  | kaist   |
| pos       | kaist   |
| lemma     | kaist   |
| depparse  | kaist   |

2022-02-14 14:20:23 INFO: Use device: cpu
2022-02-14 14:20:23 INFO: Loading: tokenize
2022-02-14 14:20:23 INFO: Loading: pos
2022-02-14 14:20:23 INFO: Loading: lemma
2022-02-14 14:20:24 INFO: Loading: depparse
2022-02-14 14:20:24 INFO: Done loading processors!


In [3]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.prepare()

def extract_noun(text):
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if pos.startswith('N'):
            yield token

# Data Load

In [4]:
data = pd.read_excel('../data/crawling/crawling.xlsx', index_col=0)
data.head()

,hotel,star,review,date,date.1
0,라까사 호텔 서울,4,접근성 좋고 괝찬은듯.,2일 전,2022. 02. 06
1,라까사 호텔 서울,4,"샤워가운 교체 시급합니다. 허리고리, 주머니가 다 떨어져있고, 실밥이 주렁주렁 달렸...",2022. 01. 30,2022. 01. 30
2,라까사 호텔 서울,5,친절해요:),2022. 01. 14,2022. 01. 14
3,라까사 호텔 서울,1,응대수준이낮음,2022. 01. 09,2022. 01. 09
4,라까사 호텔 서울,1,"최악의 크리스마스 스테이였네요.\n여름에 웨스트 더블룸에서 친구와 호캉스 했었고, ...",2022. 01. 05,2022. 01. 05


### 라까사호텔 서울

In [5]:
data1 = data.iloc[:390,:];
data1.head()

,hotel,star,review,date,date.1
0,라까사 호텔 서울,4,접근성 좋고 괝찬은듯.,2일 전,2022. 02. 06
1,라까사 호텔 서울,4,"샤워가운 교체 시급합니다. 허리고리, 주머니가 다 떨어져있고, 실밥이 주렁주렁 달렸...",2022. 01. 30,2022. 01. 30
2,라까사 호텔 서울,5,친절해요:),2022. 01. 14,2022. 01. 14
3,라까사 호텔 서울,1,응대수준이낮음,2022. 01. 09,2022. 01. 09
4,라까사 호텔 서울,1,"최악의 크리스마스 스테이였네요.\n여름에 웨스트 더블룸에서 친구와 호캉스 했었고, ...",2022. 01. 05,2022. 01. 05


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

cv1 = CountVectorizer(max_features=100, tokenizer=extract_nouns)
tdm1 = cv1.fit_transform(data1['review'].values.astype('U'))

word_count1 = pd.DataFrame({
    '단어': cv1.get_feature_names(),
    '빈도': tdm1.sum(axis=0).flat
})
word_count1.sort_values('빈도', ascending=False).head(10)

,단어,빈도
16,깔끔,43
38,방,38
9,객실,31
86,친절,30
2,~,28
62,위치,27
19,깨끗하,25
97,호텔,18
31,만족,18
82,최고,15


In [8]:
word_count1.to_excel('../data/crawling/wordcount/라까사호텔서울_단어빈도.xlsx')

In [14]:
data1_star_low = data1[data1['star']<3]
data1_star_low

,hotel,star,review,date,date.1
3,라까사 호텔 서울,1,응대수준이낮음,2022. 01. 09,2022. 01. 09
4,라까사 호텔 서울,1,"최악의 크리스마스 스테이였네요.\n여름에 웨스트 더블룸에서 친구와 호캉스 했었고, ...",2022. 01. 05,2022. 01. 05
29,라까사 호텔 서울,2,다른곳에서 느껴보지 못한 서비스 ㄴㅇㄱ\n\n스위트룸 2박 했습니다.\n청소 생각보...,2021. 10. 11,2021. 10. 11
30,라까사 호텔 서울,1,에어컨소리나서 방바꿨는데두 소리나구 욕조 없음 ㅠ,2021. 10. 05,2021. 10. 05
38,라까사 호텔 서울,2,침대 밑으로 핸드폰이 빠져서 침대 밑을 봤는데\n쓰던 콘돔 나옴,2021. 08. 22,2021. 08. 22
45,라까사 호텔 서울,1,서비스친절도,2021. 08. 02,2021. 08. 02
63,라까사 호텔 서울,2,위치 빼고 다 별로에요 정말.. 객실 관리 좀 하세요 너무 더러워요,2021. 06. 04,2021. 06. 04
72,라까사 호텔 서울,1,"더러워요, 세월의흔적이 아니에요. 관리 미흡. 매우 미흡. 다시안갈꺼같아요.",2021. 05. 20,2021. 05. 20
161,라까사 호텔 서울,2,옆방 방음이 너무 안되네요. \n새벽 한시에 옆방 투숙객 신음 소리에 잠을 설쳤습니...,2020. 11. 29,2020. 11. 29
171,라까사 호텔 서울,2,"시설이 낙후된건지 뭔지. 김영수 위원장님이 추천해줘서 그날 뵙고 숙박도 예약했는데,...",2020. 11. 18,2020. 11. 18


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

cv1 = CountVectorizer(max_features=100, tokenizer=extract_nouns)
tdm1 = cv1.fit_transform(data1_star_low['review'].values.astype('U'))

word_count1 = pd.DataFrame({
    '단어': cv1.get_feature_names(),
    '빈도': tdm1.sum(axis=0).flat
})
word_count1.sort_values('빈도', ascending=False).head(10)

,단어,빈도
51,욕조,8
22,방,6
4,객실,6
15,말,5
33,스테이,5
84,침대,5
42,예약,4
10,더블룸,4
29,서비스,4
30,소리,4


### 라까사호텔 광명

In [10]:
data2 = data.iloc[394:1386,:];
data2.head()

,hotel,star,review,date,date.1
394,라까사 호텔 광명,5,깔끔하고 방이 따뜻해서 정말 좋앗어요,7시간 전,2022. 02. 08
395,라까사 호텔 광명,5,깨끗하고 방도 따뜻했어요,3일 전,2022. 02. 05
396,라까사 호텔 광명,5,객실컨디션굿,3일 전,2022. 02. 05
397,라까사 호텔 광명,5,대부분 만족스러워요. 동네에 이정도 호텔이 있는게 정말 좋네요,3일 전,2022. 02. 05
398,라까사 호텔 광명,5,잘쉬다가가요,4일 전,2022. 02. 05


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

cv2 = CountVectorizer(max_features=100, tokenizer=extract_nouns)
tdm2 = cv2.fit_transform(data2['review'].values.astype('U'))

word_count2 = pd.DataFrame({
    '단어': cv2.get_feature_names(),
    '빈도': tdm2.sum(axis=0).flat
})
word_count2.sort_values('빈도', ascending=False).head(10)

,단어,빈도
14,깔끔,148
84,친절,114
16,깨끗하,111
3,~,99
65,이용,77
51,시설,64
66,이케아,61
97,호텔,58
77,직원분,54
30,방,51


In [11]:
word_count2.to_excel('../data/crawling/wordcount/라까사호텔광명_단어빈도.xlsx')

In [13]:
data2_star_low = data2[data2['star']<3]
data2_star_low

,hotel,star,review,date,date.1
473,라까사 호텔 광명,2,"리셉션 수준(잡담, 건성)이 호텔 격을 좌우한다는 것을 처음 깨달음. 침구류 무난하...",2021. 09. 26,2021. 09. 26
478,라까사 호텔 광명,2,객실은 미리 마운틴 뷰로 요청할 수 있다는 정보를 듣고 가서 제가 예상했던 객실 컨...,2021. 09. 22,2021. 09. 22
480,라까사 호텔 광명,2,다른 후기들처럼 청결도가 좀.... 머리카락 이곳저곳에 있었고 먼지도 많아요. 입구...,2021. 09. 20,2021. 09. 20
492,라까사 호텔 광명,1,체크인후 바로 침대에 이물질이 있고 바로 재정비 부탁드렸습니다. 하우스키퍼분께서 청...,2021. 08. 30,2021. 08. 30
544,라까사 호텔 광명,2,직원분들도 다 친절하셨고 객실도 깨끗한 편이었고 위치도 좋았는데.\n상상이상으로 더...,2021. 07. 13,2021. 07. 13
578,라까사 호텔 광명,2,생각보다 별로에요\n저렴한 기본스텐다드라서 그럴수도 있어요,2021. 06. 09,2021. 06. 09
806,라까사 호텔 광명,2,층전기 없음 빌리려면 보증금 만원\n욕실 불청결,2020. 12. 13,2020. 12. 13
833,라까사 호텔 광명,2,"인테리어나 분위기, 위치는 좋았습니다만, 객실이 너무 불청결해서 크게 실망했습니다....",2020. 11. 15,2020. 11. 15
836,라까사 호텔 광명,1,지하 주차장 밸을 눌러도 직원이 없는지 한참 기다림\n기다리다가 호탤로전화해서 들어...,2020. 11. 15,2020. 11. 15
869,라까사 호텔 광명,1,"이케아에 가까워서 선택한 숙소\n전체적인 청소상태가 불량했고,\n욕실에 물이 안빠져...",2020. 10. 12,2020. 10. 12


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

cv2 = CountVectorizer(max_features=100, tokenizer=extract_nouns)
tdm2 = cv2.fit_transform(data2_star_low['review'].values.astype('U'))

word_count2 = pd.DataFrame({
    '단어': cv2.get_feature_names(),
    '빈도': tdm2.sum(axis=0).flat
})
word_count2.sort_values('빈도', ascending=False).head(10)

,단어,빈도
88,청소,9
6,객실,9
2,2,8
34,방,8
53,수압,6
97,호텔,6
91,침대,6
85,직원,6
42,분,5
29,모텔,5


### 코리아나 호텔

In [15]:
data3 = data.iloc[1391:1877,:];
data3.head()

,hotel,star,review,date,date.1
1391,코리아나 호텔,5,위치가 너무 좋고 깨끗해요,23시간 전,2022. 02. 07
1392,코리아나 호텔,5,조아요~~~~~,1일 전,2022. 02. 07
1393,코리아나 호텔,5,아주 만족합니다.추천드립니다^^,2일 전,2022. 02. 06
1394,코리아나 호텔,5,명절 마지막날 조용하게 잘 쉬다 왔네요,3일 전,2022. 02. 05
1395,코리아나 호텔,5,명절 마지막날 조용하게 잘 쉬다 왔네요,3일 전,2022. 02. 05


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

cv3 = CountVectorizer(max_features=100, tokenizer=extract_nouns)
tdm3 = cv3.fit_transform(data3['review'].values.astype('U'))

word_count3 = pd.DataFrame({
    '단어': cv3.get_feature_names(),
    '빈도': tdm3.sum(axis=0).flat
})
word_count3.sort_values('빈도', ascending=False).head(10)

,단어,빈도
81,친절,59
28,만족,41
19,깨끗하,37
97,호텔,37
31,방,33
60,위치,33
54,오래,33
3,~,30
46,시설,28
17,깔끔,27


In [14]:
word_count3.to_excel('../data/crawling/wordcount/코리아나호텔_단어빈도.xlsx')

In [16]:
data3_star_low = data3[data3['star']<3]
data3_star_low

,hotel,star,review,date,date.1
1421,코리아나 호텔,1,오래된 호텔 특성상 조금 개선해야 할 점이 많아요 침대가 정말 푹신해서 다시 왔는데...,2022. 01. 01,2022. 01. 01
1495,코리아나 호텔,2,"위치 정말 좋고,\n직원분들 친절하세요:)!\n\n(룸 컨디션은 개선이 좀 필요하다...",2021. 09. 07,2021. 09. 07
1519,코리아나 호텔,2,우선 너무오래된 건물이라 구석구석 먼지도 많고 욕실배수도 잘안되고 따뜻한물이 미지근...,2021. 08. 03,2021. 08. 03
1775,코리아나 호텔,2,호텔에서 그것도 4성급이라는 곳에서 옆방 얘가 뛰어다니는 소리로 깰 거라고는 생각 ...,2021. 01. 31,2021. 01. 31
1778,코리아나 호텔,2,첫 인상이 좋아서 재방문했는데 이번 방은 너무 청소가 제대로 안 돼 있어서 당혹스러...,2021. 01. 30,2021. 01. 30
1864,코리아나 호텔,2,오래된곳인지 바닥카펫에 먼지가 너무 많아 아이가 비염으로 너무 힘들어했네요. 위치는...,2018. 10. 17,2018. 10. 17


In [17]:
from sklearn.feature_extraction.text import CountVectorizer

cv2 = CountVectorizer(max_features=100, tokenizer=extract_nouns)
tdm2 = cv2.fit_transform(data3_star_low['review'].values.astype('U'))

word_count2 = pd.DataFrame({
    '단어': cv2.get_feature_names(),
    '빈도': tdm2.sum(axis=0).flat
})
word_count2.sort_values('빈도', ascending=False).head(10)

,단어,빈도
32,소리,2
17,먼지,2
57,침대,2
48,점,2
47,재방문,2
42,위치,2
37,옆방,2
26,불편,2
20,물,2
63,호텔,2


### 나인브릭 호텔

In [18]:
data4 = data.iloc[1879:3784,:];
data4.head()

,hotel,star,review,date,date.1
1879,홍대 나인브릭 호텔,5,깔끔하고 좋음,7시간 전,2022. 02. 08
1880,홍대 나인브릭 호텔,5,스파 완전 잘 하고 왔어요\n객실 깨끗하고 넓어요,19시간 전,2022. 02. 07
1881,홍대 나인브릭 호텔,5,좋아여 ㅎㅎ,21시간 전,2022. 02. 07
1882,홍대 나인브릭 호텔,5,늘 가는곳입니다.,1일 전,2022. 02. 07
1883,홍대 나인브릭 호텔,3,굳굳굳굳굳,1일 전,2022. 02. 07


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

cv4 = CountVectorizer(max_features=100, tokenizer=extract_nouns)
tdm4 = cv4.fit_transform(data4['review'].values.astype('U'))

word_count4 = pd.DataFrame({
    '단어': cv4.get_feature_names(),
    '빈도': tdm4.sum(axis=0).flat
})
word_count4.sort_values('빈도', ascending=False).head(10)

,단어,빈도
12,깔끔,273
2,~,192
14,깨끗하,181
87,친절,176
98,홍대,147
30,방,146
61,위치,143
53,시설,133
24,만족,124
64,이용,117


In [17]:
word_count4.to_excel('../data/crawling/wordcount/나인브릭호텔_단어빈도.xlsx')

In [19]:
data4_star_low = data4[data4['star']<3]
data4_star_low

,hotel,star,review,date,date.1
1924,홍대 나인브릭 호텔,2,일단 지불한 금액 만큼의 퀄리티는 아닌거 같아요... 침대도 딱딱해서 저랑 친구 둘...,2022. 01. 17,2022. 01. 17
1968,홍대 나인브릭 호텔,1,청결도가 최악이에요 먼지 너무많고 화장실이랑 연결된 벽 타일은 떨어져있는데 곰팡이심...,2021. 12. 12,2021. 12. 12
1983,홍대 나인브릭 호텔,1,우선 이런식의 나쁜후기쓰는 편이 아닌데 너무 별로여서 글남깁니다\n\n11월21~2...,2021. 11. 23,2021. 11. 23
2042,홍대 나인브릭 호텔,0,바른후기 운영 정책에 의해 블라인드 조치되었습니다.,2021. 09. 26,2021. 09. 26
2086,홍대 나인브릭 호텔,1,"열받... 식당이 너무 이뻐서 조식 신청했는데 촬영? 한다고 도시락으로 받았어요, ...",2021. 08. 22,2021. 08. 22
2101,홍대 나인브릭 호텔,2,"청소하던용품 그대로 두고가져있고, 닦던 휴지도 안치우고 쌓아져있었어요.\n에어컨도 ...",2021. 08. 05,2021. 08. 05
2106,홍대 나인브릭 호텔,2,화장실천장에 곰팡이 낀게 보였어요...\n욕조에 걸쳐진 타올에도 뭐가 묻어있었음,2021. 08. 01,2021. 08. 01
2152,홍대 나인브릭 호텔,1,여자 직원분들은 친절하셨는데 남자직원분들은 잘 모르겠네요. 식사제공까지 하는 호텔이...,2021. 06. 15,2021. 06. 15
2171,홍대 나인브릭 호텔,1,설명이 기재가 잘못돼서 한시간 사용함,2021. 06. 03,2021. 06. 03
2191,홍대 나인브릭 호텔,1,방 컨디션 담배냄새 쩔어 있고 수건에서 락스 냄새 나고…\n호텔 느낌이 아닌 모텔에...,2021. 05. 23,2021. 05. 23


In [20]:
from sklearn.feature_extraction.text import CountVectorizer

cv4 = CountVectorizer(max_features=100, tokenizer=extract_nouns)
tdm4 = cv4.fit_transform(data4_star_low['review'].values.astype('U'))

word_count4 = pd.DataFrame({
    '단어': cv4.get_feature_names(),
    '빈도': tdm4.sum(axis=0).flat
})
word_count4.sort_values('빈도', ascending=False).head(10)

,단어,빈도
38,사용,10
97,호텔,9
65,이용,9
74,조식,9
80,직원분,8
42,서비스,8
7,객실,8
87,최악,8
46,시,7
89,친절,7


### 더 리버사이드 호텔

In [22]:
data5 = data.iloc[3785:4525,:];
data5.head()

,hotel,star,review,date,date.1
3785,더 리버사이드 호텔,3,강가의 여관 ;; ㅠㅠ,21시간 전,2022. 02. 07
3786,더 리버사이드 호텔,5,물을 2천원은..좀,3일 전,2022. 02. 05
3787,더 리버사이드 호텔,4,나쁘지 않음,2022. 01. 28,2022. 01. 28
3788,더 리버사이드 호텔,5,위치가 좋아요,2022. 01. 26,2022. 01. 26
3789,더 리버사이드 호텔,5,오래된 호텔이라 시설은 연식이 있지만 관리가 비교적 잘 된 편입니다,2022. 01. 26,2022. 01. 26


In [19]:
from sklearn.feature_extraction.text import CountVectorizer

cv5 = CountVectorizer(max_features=100, tokenizer=extract_nouns)
tdm5 = cv5.fit_transform(data5['review'].values.astype('U'))

word_count5 = pd.DataFrame({
    '단어': cv5.get_feature_names(),
    '빈도': tdm5.sum(axis=0).flat
})
word_count5.sort_values('빈도', ascending=False).head(10)

,단어,빈도
89,침대,61
36,방,45
9,객실,44
56,시설,39
64,위치,35
98,화장실,33
96,호텔,33
62,오래,31
38,방음,31
4,가격,29


In [20]:
word_count5.to_excel('../data/crawling/wordcount/나인브릭호텔_단어빈도.xlsx')

In [23]:
data5_star_low = data5[data5['star']<3]
data5_star_low

,hotel,star,review,date,date.1
3799,더 리버사이드 호텔,1,크리스마스 17만8천원 주고\n예약했는데 우리동네 5만원짜리\n모텔보다 꼴았습니다....,2021. 12. 26,2021. 12. 26
3807,더 리버사이드 호텔,1,최악입니다 너무너무 심각하게 더럽습니다.,2021. 12. 12,2021. 12. 12
3808,더 리버사이드 호텔,1,진짜 살다살다 이런곳은 처음이네요 솔직히 싼맛에 그냥저냥 하겠지 기대조차 안하고 왔...,2021. 12. 11,2021. 12. 11
3822,더 리버사이드 호텔,1,ㄴㄴㄴㄴㄴㄴㄴㄴ,2021. 10. 26,2021. 10. 26
3838,더 리버사이드 호텔,1,자주가는곳입니다,2021. 08. 23,2021. 08. 23
...,...,...,...,...,...
4430,더 리버사이드 호텔,2,주차장부터습하고악취엘베좁고\n다만침구는짱,2018. 06. 02,2018. 06. 02
4432,더 리버사이드 호텔,1,"정말 침대랑 티비만 있었던 곳.\n조명도 너무 어둡고, 슬리퍼도 거의 다 망가졌어요...",2018. 05. 28,2018. 05. 28
4447,더 리버사이드 호텔,1,"돈 주고 발렛 맡겼는데, 체크아웃할 때 출차가 안 되어서 한시간 이상 기다리고, 최...",2018. 04. 21,2018. 04. 21
4449,더 리버사이드 호텔,2,너무 오래되었고 객실 관리상태가 좋지않았음,2018. 04. 10,2018. 04. 10


In [24]:
from sklearn.feature_extraction.text import CountVectorizer

cv5 = CountVectorizer(max_features=100, tokenizer=extract_nouns)
tdm5 = cv5.fit_transform(data5_star_low['review'].values.astype('U'))

word_count5 = pd.DataFrame({
    '단어': cv5.get_feature_names(),
    '빈도': tdm5.sum(axis=0).flat
})
word_count5.sort_values('빈도', ascending=False).head(10)

,단어,빈도
10,객실,22
35,방,18
82,진짜,17
98,화장실,17
92,침대,17
31,모텔,16
57,시설,15
97,호텔,14
89,최악,14
36,방음,11


### 광안리 스튜디오 202 호텔

In [27]:
data6 = data.iloc[4538:4545,:];
data6.head()

,hotel,star,review,date,date.1
4538,부산 광안리 스튜디오202,5,좋아용 굳,2022. 01. 27,2022. 01. 27
4539,부산 광안리 스튜디오202,5,숙소 이쁘고 좋았어요 \n위치도 해수욕장이랑 2분거리이고 편의점도 바로앞에있어서 좋...,2021. 11. 29,2021. 11. 29
4540,부산 광안리 스튜디오202,5,바다뷰는 아니였지만 자취방 느낌처럼 너무 이뻤어요 해수욕장이랑 가까워서 좋았고 바로...,2021. 10. 24,2021. 10. 24
4541,부산 광안리 스튜디오202,1,.....,2021. 09. 11,2021. 09. 11
4542,부산 광안리 스튜디오202,5,새건물이라 그런지 엄청깔끔하고 오션뷰는 아니지만 위치는 좋아요 그리고 방내부는 사장...,2021. 08. 31,2021. 08. 31


In [28]:
from sklearn.feature_extraction.text import CountVectorizer

cv6 = CountVectorizer(max_features=100, tokenizer=extract_nouns)
tdm6 = cv6.fit_transform(data6['review'].values.astype('U'))

word_count6 = pd.DataFrame({
    '단어': cv6.get_feature_names(),
    '빈도': tdm6.sum(axis=0).flat
})
word_count6.sort_values('빈도', ascending=False).head(10)

,단어,빈도
28,숙소,4
52,해수,4
10,광안리,3
35,위치,3
48,편의점,2
46,추천,2
19,바로앞,2
39,재방문,1
30,신축,1
31,엄청,1


In [29]:
word_count6.to_excel('../data/crawling/wordcount/광안리스튜디오202호텔_단어빈도.xlsx')

In [30]:
data6_star_low = data6[data6['star']<3]
data6_star_low

,hotel,star,review,date,date.1
4541,부산 광안리 스튜디오202,1,.....,2021. 09. 11,2021. 09. 11
